In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import sys
import sys; sys.path.append("/data/jerrylee/pjt/BIGFAM.v.2.0")

from src import obj2

# GS

In [2]:
cohort = "GS"

In [3]:
# load FR-reg
def get_REL_frreg_fns(frreg_path):
    fns = os.listdir(frreg_path)
    
    fns = [fn for fn in fns if fn.split(".")[-2] == "REL"]
    return fns

frreg_path = f"/data/jerrylee/pjt/BIGFAM.v.2.0/data/GS/obj2/frreg"
fns = get_REL_frreg_fns(frreg_path)
len(fns)

40

In [6]:
# overview of inpu data
for ii, fn in enumerate(fns):
    pheno = fn.split(".")[0]
    
    # load FR-reg results
    df_frreg = pd.read_csv(f"{frreg_path}/{fn}", 
                           delim_whitespace=True)
    break

df_frreg

/tmp/ipykernel_250546/95211076.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_frreg = pd.read_csv(f"{frreg_path}/{fn}",


,DOR,relationship,sex_type,Erx,slope,se,p,n
0,1,daughter-father,FM,0.707107,0.071240,0.014383,7.592001e-07,4222
1,1,daughter-mother,FF,0.500000,0.100925,0.012174,0.000000e+00,6900
2,1,daughter-sister,FF,0.750000,0.197823,0.011438,0.000000e+00,6284
3,1,different-sex-sibling,FM,0.353553,0.180606,0.011967,0.000000e+00,7482
4,1,son-brother,MM,0.500000,0.233096,0.017551,0.000000e+00,2896
5,1,son-father,MM,0.000000,0.008010,0.020450,6.952964e-01,3398
6,1,son-mother,FM,0.707107,0.095877,0.013804,4.265255e-12,4816
7,2,daughter-father-brother,FM,0.353553,0.036619,0.033252,2.710308e-01,1050
8,2,daughter-father-sister,FF,0.250000,0.079893,0.030714,9.387126e-03,1416
9,2,daughter-mother-brother,FM,0.176777,-0.040562,0.024231,9.432678e-02,1698


# estimate $V_X$

In [5]:
df = pd.DataFrame(
    columns=[
        "pheno", "lambda", "weight", 
        "Vx", "lower_Vx", "upper_Vx"
    ]
)

for ii, fn in enumerate(tqdm(fns)):
    pheno = fn.split(".")[0]
    print(f"[{ii+1}/{len(fns)}] {pheno}")
    
    # load FR-reg results
    df_frreg = pd.read_csv(f"{frreg_path}/{fn}", 
                           delim_whitespace=True)
    try:
        # for weight in range(-2, 9):
        for weight in range(-2, 9):
            df_x = obj2.estimateX(
                df_frreg,
                n_resample=100,
                regout_bin=["DOR"],
                alpha_dicts={"type": "lambda", "weight":weight},)
        
            df.loc[len(df)] = [
                pheno, df_x["lambda"].mean(), weight,
                np.median(df_x["X"]), np.percentile(df_x["X"], 2.5), 
                np.percentile(df_x["X"], 97.5)
            ]
    except:
        print(fn)
    
    break

  0%|          | 0/40 [00:00<?, ?it/s]/tmp/ipykernel_270253/2028187205.py:13: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_frreg = pd.read_csv(f"{frreg_path}/{fn}",


[1/40] avg_dia

          Prediction Result : 
          - Vx : 0.127(0.052,0.189)
          

          Prediction Result : 
          - Vx : 0.107(0.041,0.154)
          

          Prediction Result : 
          - Vx : 0.060(0.034,0.097)
          

          Prediction Result : 
          - Vx : 0.024(0.014,0.035)
          

          Prediction Result : 
          - Vx : 0.007(0.004,0.010)
          

          Prediction Result : 
          - Vx : 0.002(0.001,0.003)
          

          Prediction Result : 
          - Vx : 0.001(0.000,0.001)
          

          Prediction Result : 
          - Vx : 0.000(0.000,0.000)
          

          Prediction Result : 
          - Vx : 0.000(0.000,0.000)
          

          Prediction Result : 
          - Vx : 0.000(0.000,0.000)
          

          Prediction Result : 
          - Vx : 0.000(0.000,0.000)
          


  0%|          | 0/40 [00:18<?, ?it/s]


# estimate $V_X$ in males and females

In [11]:
df = pd.DataFrame(
    columns=[
        "pheno", "lambda", "weight", 
        "Vx_male", "lower_male_Vx", "upper_male_Vx",
        "Vx_female", "lower_female_Vx", "upper_female_Vx",
        "r", "lower_r", "upper_r"
    ]
)

for ii, fn in enumerate(tqdm(fns)):
    pheno = fn.split(".")[0]
    print(f"[{ii+1}/{len(fns)}] {pheno}")
    
    # load FR-reg results
    df_frreg = pd.read_csv(f"{frreg_path}/{fn}", 
                           delim_whitespace=True)
    try:
        # for weight in range(-2, 9):
        for weight in range(-2, 9):
            df_x = obj2.estimateXmXfR(
                df_frreg,
                n_resample=100,
                regout_bin=["DOR"],
                alpha_dicts={"type": "lambda", "weight":weight},)
        
            df.loc[len(df)] = [
                pheno, df_x["lambda"].mean(), weight,
                np.median(df_x["Xmale"]), np.percentile(df_x["Xmale"], 2.5), 
                np.percentile(df_x["Xmale"], 97.5),
                np.median(df_x["Xfemale"]), np.percentile(df_x["Xfemale"], 2.5), 
                np.percentile(df_x["Xfemale"], 97.5),
                np.median(df_x["r"]), np.percentile(df_x["r"], 2.5), 
                np.percentile(df_x["r"], 97.5)
            ]
    except:
        print(fn)
    
    break

  0%|          | 0/40 [00:00<?, ?it/s]

/tmp/ipykernel_270253/2455887457.py:15: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_frreg = pd.read_csv(f"{frreg_path}/{fn}",


[1/40] avg_dia

          Prediction Result : 
          - Vx_male : 0.176(0.089,0.290)
          - Vx_female : 0.157(0.050,0.238)
          - r : 0.200(-0.400,0.800)
          

          Prediction Result : 
          - Vx_male : 0.127(0.067,0.182)
          - Vx_female : 0.073(0.031,0.117)
          - r : 0.200(-1.000,1.000)
          

          Prediction Result : 
          - Vx_male : 0.058(0.030,0.079)
          - Vx_female : 0.026(0.011,0.039)
          - r : 0.400(-1.000,1.000)
          

          Prediction Result : 
          - Vx_male : 0.020(0.012,0.030)
          - Vx_female : 0.008(0.004,0.014)
          - r : 1.000(-1.000,1.000)
          

          Prediction Result : 
          - Vx_male : 0.006(0.004,0.008)
          - Vx_female : 0.002(0.001,0.004)
          - r : 1.000(-1.000,1.000)
          

          Prediction Result : 
          - Vx_male : 0.002(0.001,0.002)
          - Vx_female : 0.001(0.000,0.001)
          - r : 1.000(-1.000,1.000)
          

      

  0%|          | 0/40 [05:12<?, ?it/s]

avg_dia.REL.frreg
